## NYC Health DB

**Due: Saturday, November 23 by 11:59pm**

Comments about who I worked with

In [1]:
import sqlite3
import pandas as pd

import json
from collections import defaultdict as dd

from dbga import validate_submission_format

In [2]:
def print_dict_items(my_dict, prepend_key = None):
    # Convert to standard dict type
    d = dict(my_dict)

    # Loop over the key, value pairs
    for key, value in d.items():
        # If a key to prepend was passed,
        # put it in front of the current key
        if prepend_key is not None:
            key = f'{prepend_key} : {key}'

        # If the value is another dictionary,
        # call the function recursively and pass the current key to prepend
        if type(value) == dict:
            print_dict_items(value, key)
        # Otherwise, print the key and value
        else:
            if type(value) == list:
                print(f'Key: {key}\nValues:')
                for val in value:
                    print(f'{val}\n')
                print('----------')
            else:
                print(f'Key: {key}\nValue: {value}\n----------')
    return

In [3]:
import nyc

from importlib import reload
reload(nyc)

<module 'nyc' from '/home/jovyan/Databases/04_NYCHealthInsp/nyc.py'>

In [4]:
my_sql = dd(dict)

setup = []

my_sql['setup'] = setup

In [4]:
db = nyc.NYCDB()

In [5]:
db.load_new_data()

In [6]:
db.run_query("SELECT * FROM sqlite_master;")

,type,name,tbl_name,rootpage,sql
0,table,tCuisine,tCuisine,2,CREATE TABLE tCuisine (\n cuisi...
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,tViol,tViol,4,CREATE TABLE tViol (\n viol_id ...
3,index,sqlite_autoindex_tViol_1,tViol,5,None
4,table,tAction,tAction,6,CREATE TABLE tAction (\n action...
5,table,tRest,tRest,7,CREATE TABLE tRest (\n camis IN...
6,table,tInsp,tInsp,8,CREATE TABLE tInsp (\n camis IN...
7,index,sqlite_autoindex_tInsp_1,tInsp,9,None


In [8]:
db.run_query("SELECT * FROM tRest LEFT JOIN tCuisine USING(cuisine_id) WHERE cuisine_desc IS NULL;")

,camis,dba,boro,building,street,zip,phone,cuisine_id,cuisine_desc


---

1) One of the requirements for First Normal Form (1NF) is that a candidate primary key must exist. This implies that there must be a way to uniquely identify each row - which implies that the data cannot contain any duplicate rows!

This dataset fails to meet that requirement.

**Write a query that returns the number of duplicate rows in the data.**

To avoid writing a query that takes an unnecessarily long time to run (and/or crashes your kernel), think about the minimal set of columns you need to check in order to identify duplicates. You may safely assume (or, you can write your own queries to prove these to yourself!):

- Each camis ID uniquely identifies a restaurant, and the address information for each camis ID is always the same. E.g., there may be many different Mc Donalds in New York City, but each specific location has their own unique camis ID.
  
- Each violation code always has the same violation description.
  
- The 'action' field _can_ vary by violation. For example, the action taken the first time a restaurant is cited for a violation may be different from the tenth time they are cited for the same violation.

_Hint: Double check your results by loading the original .csv file into a pandas DataFrame and using the drop_duplicates() function._

In [8]:
db.run_query("SELECT * FROM tInsp LIMIT 3;")

,camis,insp_date,insp_time,viol_id,action_id
0,40511702,06/15/2015,13:22:45,02B,1
1,40511702,11/25/2014,15:54:48,20F,1
2,50046354,10/03/2016,12:47:34,10F,1


In [9]:
db.run_query("SELECT * FROM tRest LIMIT 3;")

,camis,dba,boro,building,street,zip,phone,cuisine_id
0,30075445,MORRIS PARK BAKE SHOP,BRONX,1007,MORRIS PARK AVE,10462,7188924968,25
1,30112340,WENDY'S,BROOKLYN,469,FLATBUSH AVENUE,11225,7182875005,18
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,MANHATTAN,351,WEST 57 STREET,10019,2122452912,43


In [10]:
db.run_query("SELECT * FROM tViol LIMIT 3;")

,viol_id,viol_desc
0,02B,Hot food item not held at or above 140Âº F.
1,20F,Current letter grade card not posted.
2,10F,Non-food contact surface improperly constructe...


In [11]:
db.run_query("SELECT * FROM tAction;")

,action_id,action_desc
0,1,Violations were cited in the following area(s).
1,2,Establishment Closed by DOHMH. Violations wer...
2,3,Establishment re-opened by DOHMH
3,4,Establishment re-closed by DOHMH
4,5,No violations were recorded at the time of thi...


In [7]:
db.run_query("WITH Dupes as (SELECT camis, insp_date, insp_time, viol_id, action_id, count(*) as Instances FROM tInsp GROUP BY camis, insp_date, viol_id, action_id HAVING Instances > 1) SELECT count(*) as Duplicates FROM Dupes;")

,Duplicates
0,8


In [13]:
sql1 = "WITH Dupes as (SELECT camis, insp_date, insp_time, viol_id, action_id, count(*) as Instances FROM tInsp GROUP BY camis, insp_date, viol_id, action_id HAVING Instances > 1) SELECT count(*) as Duplicates FROM Dupes;"

In [14]:
my_sql['q1']['sql'] = sql1

---

2) Determine the second most common violation. (The top violation, by a wide margin, is not very "gross").

Next, find all restaurants that have committed this violation, along with the number of citations for each.

Do NOT hardcode any violation codes or descriptions into your query. That is, if the data were updated and rankings changed, this query would still return the correct results.

Return:

- camis
- dba
- violation code
- violation description
- number of violations

Order the results by camis, ascending.

_Hint: Spot check your results by running additional queries along the way. Write a query that sorts violations by number of occurences. Select a few rows from your final answers and run queries to verify the counts you are getting, etc._

In [8]:
db.run_query("WITH TargetViol as (SELECT viol_id, count(*) as NumViol FROM tInsp GROUP BY viol_id ORDER BY NumViol DESC LIMIT 1 OFFSET 1)SELECT camis, dba, viol_id, viol_desc, count(*) as NumViol FROM tRest LEFT JOIN tInsp USING(camis) LEFT JOIN tViol USING(viol_id) WHERE viol_id IN (SELECT viol_id FROM TargetViol) GROUP BY camis ORDER BY camis ASC;" )

,camis,dba,viol_id,viol_desc,NumViol
0,30075445,MORRIS PARK BAKE SHOP,08A,Facility not vermin proof. Harborage or condit...,1
1,30112340,WENDY'S,08A,Facility not vermin proof. Harborage or condit...,2
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,08A,Facility not vermin proof. Harborage or condit...,1
3,40356151,BRUNOS ON THE BOULEVARD,08A,Facility not vermin proof. Harborage or condit...,4
4,40356483,WILKEN'S FINE FOOD,08A,Facility not vermin proof. Harborage or condit...,1
...,...,...,...,...,...
15726,50068045,CURRY LEAF,08A,Facility not vermin proof. Harborage or condit...,1
15727,50068060,TASTE OF PUNJAB,08A,Facility not vermin proof. Harborage or condit...,1
15728,50068098,SUBWAY,08A,Facility not vermin proof. Harborage or condit...,1
15729,50068305,SONRISA AZUL,08A,Facility not vermin proof. Harborage or condit...,1


In [16]:
sql2 = "WITH TargetViol as (SELECT viol_id, count(*) as NumViol FROM tInsp GROUP BY viol_id ORDER BY NumViol DESC LIMIT 1 OFFSET 1)SELECT camis, dba, viol_id, viol_desc, count(*) as NumViol FROM tRest LEFT JOIN tInsp USING(camis) LEFT JOIN tViol USING(viol_id) WHERE viol_id IN (SELECT viol_id FROM TargetViol) GROUP BY camis ORDER BY camis ASC;"

In [17]:
my_sql['q2']['sql'] = sql2

In [11]:
db.run_query("SELECT viol_id, viol_desc, count(*) as NumViol FROM tInsp LEFT JOIN tViol USING(viol_id) GROUP BY viol_id ORDER BY NumViol DESC LIMIT 10;")

,viol_id,viol_desc,NumViol
0,10F,Non-food contact surface improperly constructe...,55772
1,08A,Facility not vermin proof. Harborage or condit...,39982
2,02G,Cold food item held above 41Âº F (smoked fish ...,27844
3,04L,Evidence of mice or live mice present in facil...,27774
4,06D,"Food contact surface not properly washed, rins...",26107
5,06C,Food not protected from potential source of co...,25951
6,10B,Plumbing not properly installed or maintained;...,22132
7,02B,Hot food item not held at or above 140Âº F.,19646
8,04N,Filth flies or food/refuse/sewage-associated (...,19472
9,04M,Live roaches present in facility's food and/or...,9012


---

3) Find all restaurants that have **never** been cited for any of the top 5 violations.

Return:

- camis
- dba
- boro

Order the results by camis, ascending.

_Hint: Since we are just asking for a list of restaurants, there should be no duplicate camis ids in your results_

In [18]:
db.run_query("WITH TargetViols as (SELECT viol_id, count(*) as NumViol FROM tInsp GROUP BY viol_id ORDER BY NumViol DESC LIMIT 5), Offenders as (SELECT camis, viol_id FROM tInsp WHERE viol_id IN (SELECT viol_id FROM TargetViols)) SELECT camis, dba, boro FROM tRest LEFT JOIN tInsp USING(camis) WHERE camis NOT IN(SELECT camis FROM Offenders) GROUP BY camis ORDER BY camis ASC;")

,camis,dba,boro
0,40359480,1 EAST 66TH STREET KITCHEN,MANHATTAN
1,40367570,NI-NI'S CAFE,STATEN ISLAND
2,40369759,NECTAR COFFEE SHOP,MANHATTAN
3,40376583,WAR MEMORIAL ICE SKATING RINK,STATEN ISLAND
4,40403988,LE CAFE (CREDIT AGRICOLE),MANHATTAN
...,...,...,...
975,50068074,THWAITES PLACE PELHAM PIZZA,BRONX
976,50068096,MADE,MANHATTAN
977,50068193,SALTWATER COFFEE,MANHATTAN
978,50068323,LITTLE FORNO,BROOKLYN


In [19]:
sql3 = "WITH TargetViols as (SELECT viol_id, count(*) as NumViol FROM tInsp GROUP BY viol_id ORDER BY NumViol DESC LIMIT 5), Offenders as (SELECT camis, viol_id FROM tInsp WHERE viol_id IN (SELECT viol_id FROM TargetViols)) SELECT camis, dba, boro FROM tRest LEFT JOIN tInsp USING(camis) WHERE camis NOT IN(SELECT camis FROM Offenders) GROUP BY camis ORDER BY camis ASC;"

In [20]:
my_sql['q3']['sql'] = sql3

---

4) How many restaurants have been cited for more than 10 violations on a single day more than once?

Only include results for those days where they had more than 10 violations.

Return:

- camis
- dba
- boro
- inspection date
- number of violations

Order the results by camis and inspection date, both ascending. 

Don't worry about whether the dates are in chronological order based on their current format. Leave them as is so our results match.

_Hint: Make sure all counts are greater than 10, and note that any restaurant listed should appear at least twice!_

In [21]:
db.run_query("WITH TargetInsp AS (SELECT camis, insp_date, count(*) as NumViols FROM tInsp LEFT JOIN tRest USING(camis) GROUP BY camis, insp_date HAVING NumViols > 10), Repeats as (SELECT *, count(*) as NumDays FROM TargetInsp GROUP BY camis HAVING NumDays >1) SELECT camis, dba, boro, insp_date, NumViols FROM TargetInsp LEFT JOIN tRest USING(camis) WHERE camis IN(SELECT camis FROM Repeats) GROUP BY camis, insp_date ;")

,camis,dba,boro,insp_date,NumViols
0,41289382,JADE ASIAN RESTAURANT,QUEENS,02/20/2016,12
1,41289382,JADE ASIAN RESTAURANT,QUEENS,08/23/2017,12
2,50001880,GANG SAN DEUL,QUEENS,05/18/2016,11
3,50001880,GANG SAN DEUL,QUEENS,06/16/2016,11
4,50035614,ELUDZ LOUNGE,BROOKLYN,03/17/2016,11
5,50035614,ELUDZ LOUNGE,BROOKLYN,09/22/2016,13


In [22]:
sql4 = "WITH TargetInsp AS (SELECT camis, insp_date, count(*) as NumViols FROM tInsp LEFT JOIN tRest USING(camis) GROUP BY camis, insp_date HAVING NumViols > 10), Repeats as (SELECT *, count(*) as NumDays FROM TargetInsp GROUP BY camis HAVING NumDays >1) SELECT camis, dba, boro, insp_date, NumViols FROM TargetInsp LEFT JOIN tRest USING(camis) WHERE camis IN(SELECT camis FROM Repeats) GROUP BY camis, insp_date ;"

In [23]:
my_sql['q4']['sql'] = sql4

---

5) For each Boro, return:


- the total number of restaurants
- the total number of violations
- the average number of violations per restaurant

Order ther results by boro, ascending.

_Hint: Watch out for integer division! Double points off this time if you forget!_

In [24]:
db.run_query("WITH BoroInfo as (SELECT boro, count(DISTINCT camis) as NumRest, count(*) as NumViol FROM tRest LEFT JOIN tInsp USING (camis) GROUP BY boro ) SELECT boro, NumRest, NumViol, (1.0*NumViol)/NumRest as AvgViol FROM BoroInfo ORDER BY boro ASC;")

,boro,NumRest,NumViol,AvgViol
0,BRONX,2320,34311,14.789224
1,BROOKLYN,6218,97630,15.701190
2,MANHATTAN,10115,156851,15.506772
3,QUEENS,5728,90817,15.854923
4,STATEN ISLAND,912,13232,14.508772


In [25]:
sql5 = "WITH BoroInfo as (SELECT boro, count(DISTINCT camis) as NumRest, count(*) as NumViol FROM tRest LEFT JOIN tInsp USING (camis) GROUP BY boro ) SELECT boro, NumRest, NumViol, (1.0*NumViol)/NumRest as AvgViol FROM BoroInfo ORDER BY boro ASC;"

In [26]:
my_sql['q5']['sql'] = sql5

---

6) Find all restaurants that have more than 10 total violations regarding 'rats' or 'mice' (total for the sum of both, not individually).

Return

- camis
- dba
- boro
- cuisine description
- number of violations

Order the results by camis, ascending.

In [27]:
db.run_query("With Rodents as (SELECT camis, viol_id, viol_desc, count(*) as NumViol FROM tInsp LEFT JOIN tViol USING(viol_id) WHERE viol_desc LIKE '%rats%' OR viol_desc LIKE '%mice%' GROUP BY camis HAVING NumViol > 10) SELECT camis, dba, boro, cuisine_desc, NumViol FROM Rodents LEFT JOIN tRest USING(camis) LEFT JOIN tInsp USING(camis) LEFT JOIN tCuisine USING (cuisine_id) GROUP BY camis ORDER BY camis ASC;")

,camis,dba,boro,cuisine_desc,NumViol
0,40366543,JARDIN DE CHINA REST,QUEENS,Chinese,11
1,40366929,LITTLE BROWN JUG,QUEENS,American,12
2,40386147,JACKS ALE HOUSE,QUEENS,American,11
3,40395152,UNCLE VANYA,MANHATTAN,Russian,11
4,40659758,ROTI BOTI RESTAURANT,QUEENS,Pakistani,11
5,40821987,MILANES SPANISH RESTAURANT,MANHATTAN,Spanish,11
6,40850055,STAR LITE DELI,MANHATTAN,Delicatessen,11
7,40861669,NEW VICTORY RESTAURANT,BROOKLYN,Chinese,17
8,40995563,LEGEND COOKHOUSE,QUEENS,Caribbean,12
9,41024190,CROCODILE LOUNGE,MANHATTAN,Pizza,12


In [28]:
sql6 = "With Rodents as (SELECT camis, viol_id, viol_desc, count(*) as NumViol FROM tInsp LEFT JOIN tViol USING(viol_id) WHERE viol_desc LIKE '%rats%' OR viol_desc LIKE '%mice%' GROUP BY camis HAVING NumViol > 10) SELECT camis, dba, boro, cuisine_desc, NumViol FROM Rodents LEFT JOIN tRest USING(camis) LEFT JOIN tInsp USING(camis) LEFT JOIN tCuisine USING (cuisine_id) GROUP BY camis ORDER BY camis ASC;"

In [29]:
my_sql['q6']['sql'] = sql6

---

7) By cuisine description, find the total number of restuarants, total number of violations involving 'rats' or 'mice', and divide the two to obtain the average number of violations per restaurant.

Return:

- cuisine description
- number of restaurants
- number of violations
- average violations per restaurant

Only include results for cuisines where the average violations per restaurant is more than 3.

Order the results by the average violations per restaurant, descending.

_Hint: Once again, watch out for integer division! Triple points off this time if you forget!_

In [30]:
db.run_query("WITH Rodents as (SELECT cuisine_id, cuisine_desc, count(*) as NumViol FROM tRest LEFT JOIN tInsp USING(camis) LEFT JOIN tCuisine USING(cuisine_id) LEFT JOIN tViol USING(viol_id) WHERE viol_desc LIKE '%rats%' OR viol_desc LIKE '%mice%' GROUP BY cuisine_desc ORDER BY cuisine_desc), TotalRests as (SELECT camis, cuisine_desc, count(DISTINCT CAMIS) as NumRests FROM tRest LEFT JOIN tCuisine USING(cuisine_id) GROUP BY cuisine_desc) SELECT cuisine_desc, NumRests, NumViol, 1.0*NumViol/NumRests as AvgViol FROM Rodents LEFT JOIN TotalRests USING(cuisine_desc) ORDER BY AvgViol DESC;")

,cuisine_desc,NumRests,NumViol,AvgViol
0,Creole,27,74,2.740741
1,Polynesian,2,5,2.500000
2,Pakistani,32,77,2.406250
3,Moroccan,14,33,2.357143
4,Soul Food,55,124,2.254545
...,...,...,...,...
78,Portuguese,9,4,0.444444
79,Hawaiian,25,10,0.400000
80,Not Listed/Not Applicable,8,3,0.375000
81,Hotdogs/Pretzels,18,6,0.333333


In [31]:
sql7 = "WITH Rodents as (SELECT cuisine_id, cuisine_desc, count(*) as NumViol FROM tRest LEFT JOIN tInsp USING(camis) LEFT JOIN tCuisine USING(cuisine_id) LEFT JOIN tViol USING(viol_id) WHERE viol_desc LIKE '%rats%' OR viol_desc LIKE '%mice%' GROUP BY cuisine_desc ORDER BY cuisine_desc), TotalRests as (SELECT camis, cuisine_desc, count(DISTINCT CAMIS) as NumRests FROM tRest LEFT JOIN tCuisine USING(cuisine_id) GROUP BY cuisine_desc ORDER BY cuisine_desc) SELECT cuisine_desc, NumRests, NumViol, 1.0*NumViol/NumRests as AvgViol FROM Rodents LEFT JOIN TotalRests USING(cuisine_desc) ORDER BY AvgViol DESC --WHERE AvgViol >3 --;"

In [32]:
my_sql['q7']['sql'] = sql7

In [12]:
db.run_query("SELECT cuisine_id, cuisine_desc, count(*) as NumViol FROM tRest LEFT JOIN tCuisine USING (cuisine_id) LEFT JOIN tInsp USING (camis) GROUP BY cuisine_desc ORDER BY NumViol DESC;")

,cuisine_id,cuisine_desc,NumViol
0,3,American,89465
1,2,Chinese,41785
2,12,"Latin (Cuban, Dominican, Puerto Rican, South &...",18853
3,8,Pizza,18524
4,1,Italian,17514
...,...,...,...
79,83,Polynesian,48
80,76,Nuts/Confectionary,40
81,84,Czech,33
82,62,Californian,29


---

8) For this query, explicitly exclude violation code '10F'. By cuisine description, compute the total number of restaurants and violations for all other violation types.

Return:

- cuisine description
- violation code
- violation description
- number of restaurants
- number of violations
- average number of violations per restaurant

Order the results by the average number of violations, descending, and return the top 10 results.

_Hint: Write some additional queries to spot check your results!_

In [38]:
db.run_query("WITH No10F as (SELECT cuisine_id, viol_id, count(*) as NumViol, count(DISTINCT camis) as NumRest from tRest LEFT JOIN tInsp USING(camis) WHERE viol_id NOT LIKE '10F' GROUP BY cuisine_id, viol_id) SELECT cuisine_desc, viol_id, viol_desc, NumRest, NumViol, 1.0*NumViol/NumRest as AvgViol FROM No10F LEFT JOIN tViol USING(viol_id) LEFT JOIN tCuisine USING(cuisine_id) ORDER BY AvgViol DESC LIMIT 10;")

,cuisine_desc,viol_id,viol_desc,NumRest,NumViol,AvgViol
0,Polynesian,08A,Facility not vermin proof. Harborage or condit...,1,6,6.000000
1,Armenian,04K,Evidence of rats or live rats present in facil...,1,5,5.000000
2,Fruits/Vegetables,04L,Evidence of mice or live mice present in facil...,3,11,3.666667
3,Portuguese,08A,Facility not vermin proof. Harborage or condit...,5,18,3.600000
4,Fruits/Vegetables,08A,Facility not vermin proof. Harborage or condit...,4,14,3.500000
5,Soul Food,08A,Facility not vermin proof. Harborage or condit...,45,157,3.488889
6,Peruvian,04L,Evidence of mice or live mice present in facil...,43,150,3.488372
7,Creole,08A,Facility not vermin proof. Harborage or condit...,23,80,3.478261
8,Peruvian,08A,Facility not vermin proof. Harborage or condit...,52,176,3.384615
9,Filipino,08A,Facility not vermin proof. Harborage or condit...,21,71,3.380952


In [39]:
sql8 = "WITH No10F as (SELECT cuisine_id, viol_id, count(*) as NumViol, count(DISTINCT camis) as NumRest from tRest LEFT JOIN tInsp USING(camis) WHERE viol_id NOT LIKE '10F' GROUP BY cuisine_id, viol_id) SELECT cuisine_desc, viol_id, viol_desc, NumRest, NumViol, 1.0*NumViol/NumRest as AvgViol FROM No10F LEFT JOIN tViol USING(viol_id) LEFT JOIN tCuisine USING(cuisine_id) ORDER BY AvgViol DESC LIMIT 10;"

In [40]:
my_sql['q8']['sql'] = sql8